In [4]:
import os
import csv
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict

In [5]:
pr_file_path="C:/apache-tomcat-7.0.34/webapps/Project/"
os.chdir('C:/Program Files/MongoDB/Server/3.2/bin')
os.system(r'mongoexport --db CustomerReviews --collection doctorReviews --type=csv --fields userId,doctorId,reviewrating >'+pr_file_path+'\mongodata_train.csv')

with open(pr_file_path+"/mongodata_train.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    with open(pr_file_path+"/mongodata_test.csv", "w",newline='') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=reader.fieldnames, delimiter=",")
        for row in reader:
            writer.writerow(row)
            
file_path = os.path.expanduser(pr_file_path+'/mongodata_test.csv')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',')

#data = Dataset.load_from_file(file_path, reader=reader)

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=4)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    
out = open(pr_file_path+'/matrixFactorizationBasedRecommendations.csv', 'w',newline='')
output=csv.writer(out)

for uid, user_ratings in top_n.items():
    output.writerow([uid, [iid for (iid, _) in user_ratings]])
    
out.close()


1 ['5', '9', '7', '11']
3 ['5', '9', '7', '11']
4 ['5', '9', '7', '11']
6 ['2', '9', '7', '11']
7 ['2', '5', '7', '11']
8 ['2', '5', '9', '11']
9 ['2', '5', '9', '7']
10 ['2', '5', '9', '7']
11 ['2', '5', '9', '7']
13 ['2', '5', '9', '7']
14 ['2', '5', '9', '7']
15 ['2', '5', '9', '7']
16 ['2', '5', '9', '7']
17 ['2', '5', '9', '7']
18 ['2', '5', '9', '7']
19 ['2', '5', '9', '7']
20 ['2', '5', '9', '7']
21 ['2', '5', '9', '7']
22 ['2', '5', '9', '7']
23 ['2', '5', '9', '7']
24 ['2', '5', '9', '7']
25 ['2', '5', '9', '7']
26 ['2', '5', '9', '7']
27 ['2', '5', '9', '7']
28 ['2', '5', '9', '7']
2 ['5', '9', '7', '11']
5 ['2', '5', '9', '7']
